<a href="https://colab.research.google.com/github/Matheus-Weil/Smart-Cities-Project/blob/main/PhotoCatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install polyline

In [ ]:
import json, urllib.request
import pprint, os, json
import polyline
import numpy as np
from math import degrees, isnan
from google.colab import drive
import urllib
import math
from google.colab import drive

In [ ]:
#Inicialização
DownLoc = '/content/data'
key = "&key="
PrevImage = []

#Distância entre fotos (metros)
Dphotos = 10

In [ ]:
#Trajeto entre Pontos
def GetPolylineCoord(origin, destiny):

    mode = "walking"

    url = 'https://maps.googleapis.com/maps/api/directions/json?origin='+origin.replace(' ','+')+"&destination="+destiny.replace(' ','+')+"&mode="+mode+key
    ur = urllib.request.urlopen(url)
    result = json.load(ur)
    pprint.pprint(result)
    coord_routes = []

    for i in range (0, len (result['routes'][0]['legs'][0]['steps'])):
        j = result['routes'][0]['legs'][0]['steps'][i]['polyline']['points']
        for k in polyline.decode(j):
            coord_routes.append(k)

    return coord_routes

#Função para angulação
def FindAngle(a, b):
    vector_1 = [a[0]-b[0], a[1]-b[1]]
    vector_2 = [0,1]
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)

    return degrees(angle)

#Função Distância
def dist(x, y):
    a = (x[0] - y[0])**2 + (x[1] - y[1])**2
    b = math.sqrt(a)
    return b

#Formatação de rota
def SetCoordHeading(coord_list):
    i = 0
    new_list = []
    while i < len(coord_list)-1:
        new_list.append((coord_list[i][0], coord_list[i][1], FindAngle(coord_list[i], coord_list[i+1])))
        i+=1
    new_list.append((coord_list[-1][0], coord_list[-1][1], FindAngle(coord_list[-2], coord_list[-1])))
    return new_list

#Objeto Json
def MetaParse(MetaUrl):
    response = urllib.request.urlopen(MetaUrl)
    jsonRaw = response.read()
    jsonData = json.loads(jsonRaw)
    #return jsonData
    print(jsonData['status'])
    if jsonData['status'] == "OK":

        if 'date' in jsonData:
            return (jsonData['date'],jsonData['pano_id']) #sometimes it does not have a date!
        else:
            return (None,jsonData['pano_id'])
    else:

        return (None,None)

#Captura de Foto
def GetStreetLL(Lat,Lon,Head,File,SaveLoc):
    base = r"https://maps.googleapis.com/maps/api/streetview"
    size = r"?size=1200x800&fov=60&location="
    end = str(Lat) + "," + str(Lon) + "&heading=" + str(Head) + key
    MyUrl = base + size + end
    fi = File + ".jpg"
    MetaUrl = base + r"/metadata" + size + end
    #print MyUrl, MetaUrl #can check out image in browser to adjust size, fov to needs
    met_lis = list(MetaParse(MetaUrl))                           #nao pega imagens se nao te data
    if (met_lis[1],Head) not in PrevImage and met_lis[0] is not None:   #PrevImage is global list
        urllib.request.urlretrieve(MyUrl, os.path.join(SaveLoc,fi))
        met_lis.append(fi)
        PrevImage.append((met_lis[1],Head)) #append nova Pano ID na lista de imagens
    else:
        met_lis.append(None)
    return met_lis

In [ ]:
#Foto de Rua lado Direito
DownLoc = '/content/data'
Ini = [-25.42625795070118, -49.25913158114752]
Fim = [-25.427595111697098, -49.24361768421039]
ang = 0
print(ang)
d = dist(Ini, Fim)
print(int(d*100000))
d = int(d*100000) / Dphotos
d = int(d)
#Número de fotos
print(d)

LonPlus = (Fim [0] - Ini[0]) / d
LatPlus = (Fim [1] - Ini[1]) / d
DataList = np.zeros((d+1,3))

for i in range(d+1):
  DataList[i][0] = Ini[0] + (LonPlus * i)
  DataList[i][1] = Ini[1] + (LatPlus * i)
  DataList[i][2] = ang+180

print(DataList)

image_list = []

ct = 0
for i in DataList:
    ct += 1
    fi = "Image_" + str(ct)
    temp = GetStreetLL(Lat=i[0],Lon=i[1],Head=i[2],File=fi,SaveLoc=DownLoc)

    if temp[2] is not None:
        image_list.append(temp)

0
1557
155
[[-25.42625795 -49.25913158 180.        ]
 [-25.42626658 -49.25903149 180.        ]
 [-25.4262752  -49.2589314  180.        ]
 [-25.42628383 -49.25883131 180.        ]
 [-25.42629246 -49.25873122 180.        ]
 [-25.42630108 -49.25863113 180.        ]
 [-25.42630971 -49.25853104 180.        ]
 [-25.42631834 -49.25843095 180.        ]
 [-25.42632697 -49.25833086 180.        ]
 [-25.42633559 -49.25823077 180.        ]
 [-25.42634422 -49.25813068 180.        ]
 [-25.42635285 -49.25803059 180.        ]
 [-25.42636147 -49.25793051 180.        ]
 [-25.4263701  -49.25783042 180.        ]
 [-25.42637873 -49.25773033 180.        ]
 [-25.42638735 -49.25763024 180.        ]
 [-25.42639598 -49.25753015 180.        ]
 [-25.42640461 -49.25743006 180.        ]
 [-25.42641323 -49.25732997 180.        ]
 [-25.42642186 -49.25722988 180.        ]
 [-25.42643049 -49.25712979 180.        ]
 [-25.42643911 -49.2570297  180.        ]
 [-25.42644774 -49.25692961 180.        ]
 [-25.42645637 -49.2568

In [ ]:
#Foto de Rua lado Esquerdo
DownLoc = '/content/data2'
ang = ang +180
print(ang)
d = dist(Ini, Fim)
print(int(d*100000))
d = int(d*100000) / Dphotos
d = int(d)
#Número de fotos
print(d)

LonPlus = (Fim [0] - Ini[0]) / d
LatPlus = (Fim [1] - Ini[1]) / d
DataList = np.zeros((d+1,3))

for i in range(d+1):
  DataList[i][0] = Ini[0] + (LonPlus * i)
  DataList[i][1] = Ini[1] + (LatPlus * i)
  DataList[i][2] = ang+180

print(DataList)

image_list = []

ct = 0
for i in DataList:
    ct += 1
    fi = "Image_" + str(ct)
    temp = GetStreetLL(Lat=i[0],Lon=i[1],Head=i[2],File=fi,SaveLoc=DownLoc)

    if temp[2] is not None:
        image_list.append(temp)

180
1557
155
[[-25.42625795 -49.25913158 360.        ]
 [-25.42626658 -49.25903149 360.        ]
 [-25.4262752  -49.2589314  360.        ]
 [-25.42628383 -49.25883131 360.        ]
 [-25.42629246 -49.25873122 360.        ]
 [-25.42630108 -49.25863113 360.        ]
 [-25.42630971 -49.25853104 360.        ]
 [-25.42631834 -49.25843095 360.        ]
 [-25.42632697 -49.25833086 360.        ]
 [-25.42633559 -49.25823077 360.        ]
 [-25.42634422 -49.25813068 360.        ]
 [-25.42635285 -49.25803059 360.        ]
 [-25.42636147 -49.25793051 360.        ]
 [-25.4263701  -49.25783042 360.        ]
 [-25.42637873 -49.25773033 360.        ]
 [-25.42638735 -49.25763024 360.        ]
 [-25.42639598 -49.25753015 360.        ]
 [-25.42640461 -49.25743006 360.        ]
 [-25.42641323 -49.25732997 360.        ]
 [-25.42642186 -49.25722988 360.        ]
 [-25.42643049 -49.25712979 360.        ]
 [-25.42643911 -49.2570297  360.        ]
 [-25.42644774 -49.25692961 360.        ]
 [-25.42645637 -49.25

In [ ]:
#Rotina para salvas as fotos
DataList = [i for i in SetCoordHeading(GetPolylineCoord(start, finish)) if not isnan(i[2])]
pprint.pprint(DataList)

image_list = []

ct = 0
for i in DataList:
    ct += 1
    fi = "Image_" + str(ct)
    temp = GetStreetLL(Lat=i[0],Lon=i[1],Head=i[2],File=fi,SaveLoc=DownLoc)

    if temp[2] is not None:
        image_list.append(temp)

{'error_message': 'You must enable Billing on the Google Cloud Project at '
                  'https://console.cloud.google.com/project/_/billing/enable '
                  'Learn more at '
                  'https://developers.google.com/maps/gmp-get-started',
 'routes': [],
 'status': 'REQUEST_DENIED'}


IndexError: list index out of range